In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pyproj
import fiona
import geopandas

PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
lic_comm = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/9b002e32-0ec9-40fc-9292-cddeae4416c6", encoding="utf-8", sep=";")

C:\Users\Cosima\AppData\Local\Temp\ipykernel_15476\1859561693.py:1: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  lic_comm = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/9b002e32-0ec9-40fc-9292-cddeae4416c6", encoding="utf-8", sep=";")


In [193]:
print(lic_comm)

            Code Commune                  Commune          Code QPV Nom QPV  \
0                   1001  L'Abergement-Clémenciat               CSZ     NaN   
1                   1001  L'Abergement-Clémenciat               CSZ     NaN   
2                   1001  L'Abergement-Clémenciat               CSZ     NaN   
3                   1001  L'Abergement-Clémenciat               CSZ     NaN   
4                   1001  L'Abergement-Clémenciat               CSZ     NaN   
...                  ...                      ...               ...     ...   
939323  NR - Non réparti                      NaN  NR - Non réparti     NaN   
939324  NR - Non réparti                      NaN  NR - Non réparti     NaN   
939325  NR - Non réparti                      NaN  NR - Non réparti     NaN   
939326  NR - Non réparti                      NaN  NR - Non réparti     NaN   
939327  NR - Non réparti                      NaN  NR - Non réparti     NaN   

       Département                Région    Statut 

In [3]:
lic_comm.head(5)

,Code Commune,Commune,Code QPV,Nom QPV,Département,Région,Statut géo,Code,Fédération,F - 1 à 4 ans,...,H - NR,NR - 5 à 9 ans,NR - 10 à 14 ans,NR - 15 à 19 ans,NR - 30 à 34 ans,NR - 40 à 44 ans,NR - 45 à 49 ans,NR - 70 à 74 ans,NR - NR,Total
0,1001,L'Abergement-Clémenciat,CSZ,NaN,1,Auvergne-Rhône-Alpes,1.Champ geoc,101,FF d'Athlétisme,0,...,0,0,0,0,0,0,0,0,0,8
1,1001,L'Abergement-Clémenciat,CSZ,NaN,1,Auvergne-Rhône-Alpes,1.Champ geoc,103,FF de Badminton,0,...,0,0,0,0,0,0,0,0,0,11
2,1001,L'Abergement-Clémenciat,CSZ,NaN,1,Auvergne-Rhône-Alpes,1.Champ geoc,105,FF de Basketball,0,...,0,0,0,0,0,0,0,0,0,1
3,1001,L'Abergement-Clémenciat,CSZ,NaN,1,Auvergne-Rhône-Alpes,1.Champ geoc,108,FF de Cyclisme,0,...,0,0,0,0,0,0,0,0,0,2
4,1001,L'Abergement-Clémenciat,CSZ,NaN,1,Auvergne-Rhône-Alpes,1.Champ geoc,109,FF d'Équitation,0,...,0,0,0,0,0,0,0,0,0,10


In [171]:
lic_comm.columns

Index(['Code Commune', 'Commune', 'Code QPV', 'Nom QPV', 'Département',
       'Région', 'Statut géo', 'Code', 'Fédération', 'F - 1 à 4 ans',
       'F - 5 à 9 ans', 'F - 10 à 14 ans', 'F - 15 à 19 ans',
       'F - 20 à 24 ans', 'F - 25 à 29 ans', 'F - 30 à 34 ans',
       'F - 35 à 39 ans', 'F - 40 à 44 ans', 'F - 45 à 49 ans',
       'F - 50 à 54 ans', 'F - 55 à 59 ans', 'F - 60 à 64 ans',
       'F - 65 à 69 ans', 'F - 70 à 74 ans', 'F - 75 à 79 ans',
       'F - 80 à 99 ans', 'F - NR', 'H - 1 à 4 ans', 'H - 5 à 9 ans',
       'H - 10 à 14 ans', 'H - 15 à 19 ans', 'H - 20 à 24 ans',
       'H - 25 à 29 ans', 'H - 30 à 34 ans', 'H - 35 à 39 ans',
       'H - 40 à 44 ans', 'H - 45 à 49 ans', 'H - 50 à 54 ans',
       'H - 55 à 59 ans', 'H - 60 à 64 ans', 'H - 65 à 69 ans',
       'H - 70 à 74 ans', 'H - 75 à 79 ans', 'H - 80 à 99 ans', 'H - NR',
       'NR - 5 à 9 ans', 'NR - 10 à 14 ans', 'NR - 15 à 19 ans',
       'NR - 30 à 34 ans', 'NR - 40 à 44 ans', 'NR - 45 à 49 ans',
       '

###  Verification et Conversion des types des donnees###

In [4]:
print(lic_comm.shape)# la base compte 68 variables au total
#lic_comm=lic_comm.drop(columns=['l_qp_2019','l_qp_f_2019','l_qp_h_2019'])# variales non pertinentes pour notre étude
#print(lic_comm.shape)# la base compte 68 variables au total
lic_comm.dtypes

(939328, 54)


Code Commune        object
Commune             object
Code QPV            object
Nom QPV             object
Département         object
Région              object
Statut géo          object
Code                 int64
Fédération          object
F - 1 à 4 ans        int64
F - 5 à 9 ans        int64
F - 10 à 14 ans      int64
F - 15 à 19 ans      int64
F - 20 à 24 ans      int64
F - 25 à 29 ans      int64
F - 30 à 34 ans      int64
F - 35 à 39 ans      int64
F - 40 à 44 ans      int64
F - 45 à 49 ans      int64
F - 50 à 54 ans      int64
F - 55 à 59 ans      int64
F - 60 à 64 ans      int64
F - 65 à 69 ans      int64
F - 70 à 74 ans      int64
F - 75 à 79 ans      int64
F - 80 à 99 ans      int64
F - NR               int64
H - 1 à 4 ans        int64
H - 5 à 9 ans        int64
H - 10 à 14 ans      int64
H - 15 à 19 ans      int64
H - 20 à 24 ans      int64
H - 25 à 29 ans      int64
H - 30 à 34 ans      int64
H - 35 à 39 ans      int64
H - 40 à 44 ans      int64
H - 45 à 49 ans      int64
H

In [9]:
lic_comm['Code']=lic_comm['Code'].astype('string') #modification du type code
lic_comm['Code Commune'] = lic_comm['Code Commune'].astype('string')
lic_comm['Commune'] = lic_comm['Commune'].astype('string')
lic_comm['Département'] = lic_comm['Département'].astype('string')
lic_comm['Région'] = lic_comm['Région'].astype('string')
lic_comm['Statut géo'] = lic_comm['Statut géo'].astype('string')
lic_comm['Fédération'] = lic_comm['Fédération'].astype('string')
lic_comm['Code QPV'] = lic_comm['Code QPV'].astype('string')
lic_comm['Nom QPV'] = lic_comm['Nom QPV'].astype('string')
print(lic_comm.dtypes)

Code Commune        string[python]
Commune             string[python]
Code QPV            string[python]
Nom QPV             string[python]
Département         string[python]
Région              string[python]
Statut géo          string[python]
Code                string[python]
Fédération          string[python]
F - 1 à 4 ans                int64
F - 5 à 9 ans                int64
F - 10 à 14 ans              int64
F - 15 à 19 ans              int64
F - 20 à 24 ans              int64
F - 25 à 29 ans              int64
F - 30 à 34 ans              int64
F - 35 à 39 ans              int64
F - 40 à 44 ans              int64
F - 45 à 49 ans              int64
F - 50 à 54 ans              int64
F - 55 à 59 ans              int64
F - 60 à 64 ans              int64
F - 65 à 69 ans              int64
F - 70 à 74 ans              int64
F - 75 à 79 ans              int64
F - 80 à 99 ans              int64
F - NR                       int64
H - 1 à 4 ans                int64
H - 5 à 9 ans       

In [10]:
print(lic_comm.shape)

(939328, 55)


In [11]:
print(lic_comm['Code'].nunique())# 110 federation 
print(lic_comm['Fédération'].nunique())# 110 federation
print(lic_comm['Département'].nunique())#206 dep
print(lic_comm['Code Commune'].nunique())#34900 communes
print(lic_comm['Code Commune'].nunique())
print(lic_comm['Nom QPV'].nunique()) #1304

110
110
119
34897
34897
1304


#### vérifions s'il y'a des valeurs manquantes

Seules les variables 'Commune' et 'Nom QPV' comprennent des valeurs manquantes. La variable QPV est remplie seulement pour les communes au sein desquelles il y a un quartier prioritaire des politiques de la ville, ce qui ne concerne pas toutes les communes. Il est donc normal qu'il y ait des valeurs manuantes pour ces variables.

In [176]:
lic_comm.isna().any()

Code Commune        False
Commune              True
Code QPV            False
Nom QPV              True
Département         False
Région              False
Statut géo          False
Code                False
Fédération          False
F - 1 à 4 ans       False
F - 5 à 9 ans       False
F - 10 à 14 ans     False
F - 15 à 19 ans     False
F - 20 à 24 ans     False
F - 25 à 29 ans     False
F - 30 à 34 ans     False
F - 35 à 39 ans     False
F - 40 à 44 ans     False
F - 45 à 49 ans     False
F - 50 à 54 ans     False
F - 55 à 59 ans     False
F - 60 à 64 ans     False
F - 65 à 69 ans     False
F - 70 à 74 ans     False
F - 75 à 79 ans     False
F - 80 à 99 ans     False
F - NR              False
H - 1 à 4 ans       False
H - 5 à 9 ans       False
H - 10 à 14 ans     False
H - 15 à 19 ans     False
H - 20 à 24 ans     False
H - 25 à 29 ans     False
H - 30 à 34 ans     False
H - 35 à 39 ans     False
H - 40 à 44 ans     False
H - 45 à 49 ans     False
H - 50 à 54 ans     False
H - 55 à 59 

# analyse exploratoire#

In [177]:
lic_comm.describe()

,F - 1 à 4 ans,F - 5 à 9 ans,F - 10 à 14 ans,F - 15 à 19 ans,F - 20 à 24 ans,F - 25 à 29 ans,F - 30 à 34 ans,F - 35 à 39 ans,F - 40 à 44 ans,F - 45 à 49 ans,...,H - NR,NR - 5 à 9 ans,NR - 10 à 14 ans,NR - 15 à 19 ans,NR - 30 à 34 ans,NR - 40 à 44 ans,NR - 45 à 49 ans,NR - 70 à 74 ans,NR - NR,Total
count,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,...,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000,939328.000000
mean,0.167556,1.138687,1.648449,0.761686,0.239381,0.216534,0.210617,0.243640,0.259474,0.269921,...,0.014363,0.364538,0.081176,0.000099,0.000001,0.000001,0.000002,0.000002,0.036939,17.388674
std,2.818289,14.069227,15.074177,8.389632,3.585600,1.869779,1.713701,1.939336,1.980212,2.139287,...,2.769538,10.700431,2.382278,0.010266,0.001032,0.001032,0.001459,0.001459,2.566003,103.985309
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000
max,1009.000000,4935.000000,3743.000000,1775.000000,1307.000000,406.000000,478.000000,751.000000,727.000000,574.000000,...,2535.000000,3634.000000,789.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1851.000000,33194.000000


In [218]:
lic_comm_sup=lic_comm.drop_duplicates(subset=['Région','Département','Code Commune','Code']).reset_index(drop=True)

In [219]:
print(lic_comm_sup)

            Code Commune                  Commune          Code QPV Nom QPV  \
0                   1001  L'Abergement-Clémenciat               CSZ     NaN   
1                   1001  L'Abergement-Clémenciat               CSZ     NaN   
2                   1001  L'Abergement-Clémenciat               CSZ     NaN   
3                   1001  L'Abergement-Clémenciat               CSZ     NaN   
4                   1001  L'Abergement-Clémenciat               CSZ     NaN   
...                  ...                      ...               ...     ...   
887150  NR - Non réparti                      NaN  NR - Non réparti     NaN   
887151  NR - Non réparti                      NaN  NR - Non réparti     NaN   
887152  NR - Non réparti                      NaN  NR - Non réparti     NaN   
887153  NR - Non réparti                      NaN  NR - Non réparti     NaN   
887154  NR - Non réparti                      NaN  NR - Non réparti     NaN   

       Département                Région    Statut 

sans zonage 887155

In [133]:
#a=lic_comm_sup[['Code QPV','Code Commune']]
#print(a.shape)
#a.drop_duplicates()
#print(a.shape)
#h=lic_comm.drop_duplicates(['Région','Département','Code Commune','Code','Code QPV'])
#print(h.shape)

(887155, 2)
(887155, 2)
(887155, 54)


### contruction des ases par sexe et par age##

In [ ]:
def donnees_par_sexe(base_initial):
    base=base_initial[base_initial.columns.difference([ 'Code QPV', 'Nom QPV','Statut géo'])]
    base.groupy(['Code Commune', 'Commune', 'Département','Région', 'Statut géo', 'Code', 'Fédération'])
    

**** taux de licences par regions****
A echelle Regionnale:
 taux de licence par sexe
 sport plus pratiqué par les par sexe
 

In [262]:
#proportion_licencie_sexe=lic_comm[['Région','F - NR','H - NR','NR - NR','Total']].groupby('Région').sum()
#proportion_licencie_sexe

def lic_reg_per_sex(base_initial):
    base=base_initial[base_initial.columns.difference(['Code Commune', 'Commune', 'Code QPV', 'Nom QPV', 'Département',
                                    'Statut géo', 'Code', 'Fédération'])]
    base.groupby('Région').sum()
    base['F']=base[['F - 5 à 9 ans', 'F - 10 à 14 ans', 'F - 15 à 19 ans',
              'F - 20 à 24 ans', 'F - 25 à 29 ans', 'F - 30 à 34 ans',
              'F - 35 à 39 ans', 'F - 40 à 44 ans', 'F - 45 à 49 ans',
              'F - 50 à 54 ans', 'F - 55 à 59 ans', 'F - 60 à 64 ans',
              'F - 65 à 69 ans', 'F - 70 à 74 ans', 'F - 75 à 79 ans',
              'F - 80 à 99 ans', 'F - NR']].sum(axis=1)
    base['H']=base[['H - 1 à 4 ans', 'H - 5 à 9 ans',
              'H - 10 à 14 ans', 'H - 15 à 19 ans', 'H - 20 à 24 ans',
              'H - 25 à 29 ans', 'H - 30 à 34 ans', 'H - 35 à 39 ans',
              'H - 40 à 44 ans', 'H - 45 à 49 ans', 'H - 50 à 54 ans',
              'H - 55 à 59 ans', 'H - 60 à 64 ans', 'H - 65 à 69 ans',
              'H - 70 à 74 ans', 'H - 75 à 79 ans', 'H - 80 à 99 ans', 'H - NR']].sum(axis=1)
    base['NR']=base[['NR - 5 à 9 ans', 'NR - 10 à 14 ans', 'NR - 15 à 19 ans',
                 'NR - 30 à 34 ans', 'NR - 40 à 44 ans', 'NR - 45 à 49 ans',
                 'NR - 70 à 74 ans', 'NR - NR']].sum(axis=1)
    base1=base[['Région','H','F','NR','Total']].groupby('Région').sum().reset_index()
    base2=base1
    base2['Total_m']=base2['H']+base2['F']+base2['NR']
    base2['taux_F']=(base2['F']/base2['Total_m'])* 100
    base2['taux_H']=(base2['H']/base2['Total_m'])* 100
    base2['taux_NR']=(base2['NR']/base2['Total_m'])* 100
    return base2


In [263]:
taux_licencies_par_sexe=lic_reg_per_sex(lic_comm_sup)
taux_licencies_par_sexe

C:\Users\user\AppData\Local\Temp\ipykernel_17692\4253158211.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base['F']=base[['F - 5 à 9 ans', 'F - 10 à 14 ans', 'F - 15 à 19 ans',
C:\Users\user\AppData\Local\Temp\ipykernel_17692\4253158211.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base['H']=base[['H - 1 à 4 ans', 'H - 5 à 9 ans',
C:\Users\user\AppData\Local\Temp\ipykernel_17692\4253158211.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,Région,H,F,NR,Total,Total_m,taux_F,taux_H,taux_NR
0,Auvergne-Rhône-Alpes,1243526,791839,53069,2107406,2088434,37.915443,59.543467,2.541091
1,Bourgogne-Franche-Comté,374571,239393,4445,625090,618409,38.711112,60.570108,0.718780
2,Bretagne,550662,351233,71146,980595,973041,36.096423,56.591860,7.311717
3,COM,41668,26931,102,69669,68701,39.200303,60.651228,0.148469
4,Centre-Val de Loire,373228,244180,14692,638650,632100,38.629964,59.045721,2.324316
5,Corse,48064,24307,260,73491,72631,33.466426,66.175600,0.357974
6,DROM,200454,110739,1161,316305,312354,35.453044,64.175263,0.371694
7,Etranger,50237,29205,2025,81824,81467,35.848871,61.665460,2.485669
8,Grand Est,747696,472985,21883,1256523,1242564,38.065243,60.173641,1.761117
9,Hauts-de-France,746740,435379,56015,1252889,1238134,35.164126,60.311727,4.524147


*** representation graphique**

In [264]:
import json

In [266]:
test=taux_licencies_par_sexe[['Région','taux_F','taux_H','taux_NR']]
test_json=test.to_json(orient='records')
test_json

str

In [ ]:
import plotly.express as px
import folium
import branca

In [ ]:
def carte_taux_interractive(base):
    #initialisation de la carte de france
    geomap_france=folium.Map(location=[48.8566, 2.3522], zoom_start=6, tiles="OpenStreetMap" )
    # ajout des marquers avec popup graphiques  pour chaque region
    for i , ligne in base.iterows():
        nom_region=ligne['Région']

        #créer le graphique plotly pour la région
        fig_region=px.bar(
            pd.date_range(ligne).transpose(),
            x=['Femmes','Hommes','Non Répartis'],
            y=['taux_F','taux_H','taux_NR'],
            title=f'taux de licenciés par sexe - {nom_region}'
        )

        #enregistrer le graphe au format HTML
        chemin_html_region=f'{nom_region}_graph.html'
        fig_region.write_html(chemin_html_region)
        
        #lire le fichier html
        with open(chemin_html_region,encoding='utf-8') as f:
            html_region=f.read()
        
        #coordonnees pour le marqueur
        coor_region

        #creerle popup
        iframe_region=branca.element.IFrame(html=html_region,width=500,height=300)
        popup_region=folium.Popup(iframe_region,max_width=500)

        #ajouter le marqueur à la carte
        folium.Marker(location=coor_region,popup=popup_region).add_to(geomap_france)
    #fin
    return geomap_france

**** analyse au niveu departement******

In [14]:
lic__pop=lic_comm.drop(columns=pop_object)# supprimer les varaibles inutilisées
print(lic__pop)

       code_commune                libelle                region fed_2019  \
0              1001  Abergement-Clémenciat  Auvergne-Rhône-Alpes      101   
1              1001  Abergement-Clémenciat  Auvergne-Rhône-Alpes      103   
2              1001  Abergement-Clémenciat  Auvergne-Rhône-Alpes      105   
3              1001  Abergement-Clémenciat  Auvergne-Rhône-Alpes      108   
4              1001  Abergement-Clémenciat  Auvergne-Rhône-Alpes      109   
...             ...                    ...                   ...      ...   
864712        97617               TSINGONI               Mayotte      258   
864713        97617               TSINGONI               Mayotte      261   
864714        97617               TSINGONI               Mayotte      401   
864715        97617               TSINGONI               Mayotte      403   
864716        97617               TSINGONI               Mayotte      604   

                                                  nom_fed  l_2019  l_0_4_20

importation de la base des dep puis jonture avec la base des départements

In [267]:
dep=pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/dbe8a621-a9c4-4bc3-9cae-be1699c5ff25")
dep.head(10)


,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes
5,1007,AMBRONAY,1500,AMBRONAY,NaN,46.005591,5.357607,7.0,NaN,Ambronay,Ambronay,1,Ain,84.0,Auvergne-Rhône-Alpes
6,1008,AMBUTRIX,1500,AMBUTRIX,NaN,45.936713,5.332809,8.0,NaN,Ambutrix,Ambutrix,1,Ain,84.0,Auvergne-Rhône-Alpes
7,1009,ANDERT ET CONDON,1300,ANDERT ET CONDON,NaN,45.787357,5.657883,9.0,NaN,Andert-et-Condon,Andert-et-Condon,1,Ain,84.0,Auvergne-Rhône-Alpes
8,1010,ANGLEFORT,1350,ANGLEFORT,NaN,45.909372,5.795160,10.0,NaN,Anglefort,Anglefort,1,Ain,84.0,Auvergne-Rhône-Alpes
9,1011,APREMONT,1100,APREMONT,NaN,46.205498,5.657815,11.0,NaN,Apremont,Apremont,1,Ain,84.0,Auvergne-Rhône-Alpes


In [16]:
dep.dtypes

code_commune_INSEE       object
nom_commune_postal       object
code_postal               int64
libelle_acheminement     object
ligne_5                  object
latitude                float64
longitude               float64
code_commune            float64
article                  object
nom_commune              object
nom_commune_complet      object
code_departement         object
nom_departement          object
code_region             float64
nom_region               object
dtype: object

In [17]:
lic_dep=pd.merge(lic__pop, dep[['code_commune_INSEE','latitude','longitude','code_departement','nom_departement']], 
                 left_on="code_commune", right_on="code_commune_INSEE")

In [18]:
lic_dep.head(5)

,code_commune,libelle,region,fed_2019,nom_fed,l_2019,l_0_4_2019,l_5_9_2019,l_10_14_2019,l_15_19_2019,...,pop_45_59_2018,pop_60_74_2018,pop_75_2018,popf_2018,poph_2018,code_commune_INSEE,latitude,longitude,code_departement,nom_departement
0,29141,Loqueffret,Bretagne,111,FF de football,2,0,0,1,0,...,67.0,98.0,46.0,200.0,143.0,29141,48.329497,-3.859169,29,Finistère
1,29141,Loqueffret,Bretagne,115,FF de handball,3,0,0,0,2,...,67.0,98.0,46.0,200.0,143.0,29141,48.329497,-3.859169,29,Finistère
2,29141,Loqueffret,Bretagne,117,FF de judo-jujitsu et disciplines associées,2,0,0,1,0,...,67.0,98.0,46.0,200.0,143.0,29141,48.329497,-3.859169,29,Finistère
3,29141,Loqueffret,Bretagne,118,FF de lutte et disciplines associées,3,0,1,1,0,...,67.0,98.0,46.0,200.0,143.0,29141,48.329497,-3.859169,29,Finistère
4,29141,Loqueffret,Bretagne,119,FF de natation,1,0,0,0,0,...,67.0,98.0,46.0,200.0,143.0,29141,48.329497,-3.859169,29,Finistère
